# Tensorflow

In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.simplefilter('ignore', RuntimeWarning)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data = fetch_california_housing()

We work with TF1 but ask it to emulate TF2 behavior

In [ ]:
# import tensorflow.compat.v2 as tf
# tf.enable_v2_behavior()
import tensorflow as tf

In [ ]:
%%capture
import tensorflow_probability as tfp
tfd = tfp.distributions

## Working with tensors

Almost exaclty like numpy arrays.`m

In [ ]:
tf.constant([1., 2., 3.])

In [ ]:
x = tf.Variable([[1.,2.,3.], [4.,5.,6.]])

In [ ]:
x.shape

In [ ]:
x.dtype

### Conversin to numpy

In [ ]:
x.numpy()

### Indexing

In [ ]:
x[:, :2]

### Assignment

In [ ]:
x[0,:].assign([3.,2.,1.])

In [ ]:
x

### Reductions

In [ ]:
tf.reduce_mean(x, axis=0)

In [ ]:
tf.reduce_sum(x, axis=1)

### Broadcasting

In [ ]:
x + 10

In [ ]:
x * 10

In [ ]:
x - tf.reduce_mean(x, axis=1)[:, tf.newaxis]

### Matrix operations

In [ ]:
x @ tf.transpose(x)

### Ufuncs

In [ ]:
tf.exp(x)

In [ ]:
tf.sqrt(x)

### Random numbers

In [ ]:
X = tf.random.normal(shape=(10,4))
y = tf.random.normal(shape=(10,1))

### Linear algebra

In [ ]:
tf.linalg.lstsq(X, y)

### Vectorization

In [ ]:
X = tf.random.normal(shape=(1000,10,4))
y = tf.random.normal(shape=(1000,10,1))

In [ ]:
tf.linalg.lstsq(X, y)

### Automatic differntiation

In [ ]:
def f(x,y):
    return x**2 + 2*y**2 + 3*x*y

#### Gradient

In [ ]:
x, y = tf.Variable(1.0), tf.Variable(2.0)

In [ ]:
with tf.GradientTape() as tape:
    z = f(x, y)

In [ ]:
tape.gradient(z, [x,y])

#### Hessian

In [ ]:
with tf.GradientTape(persistent=True) as H_tape:
    with tf.GradientTape() as J_tape:
        z = f(x, y)
    Js = J_tape.gradient(z, [x,y])
Hs = [H_tape.gradient(J, [x,y]) for J in Js]
del H_tape                    

In [ ]:
np.array(Hs)

## Keras

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target)

In [ ]:
y_train.min(), y_train.max()

In [ ]:
scalar = StandardScaler()
X_train_s = scalar.fit_transform(X_train)
X_test_s = scalar.transform(X_test)

In [ ]:
import tensorflow.keras as keras

In [ ]:
Dense = keras.layers.Dense

We can consider a DL model as just a black box with a bunch of unnown parameters. For exanple, when the outoput is a Dense layer with just one node, the entire network model is just doing some form of regression. Hence we can replace a linear regression model with such a neural network model and run MCMC or VI as usual.

In [ ]:
model = keras.models.Sequential([
    Dense(30, 
          activation='elu', 
          input_shape=X_train.shape[1:]),
    Dense(1)
])

In [ ]:
model.compile(loss="mse", optimizer="nadam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
model.layers[0].name

In [ ]:
model.layers[0].activation

In [ ]:
hist = model.fit(X_train_s, 
                 y_train,
                 epochs=10,
                 validation_split=0.2)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(hist.history)

In [ ]:
df.head()

In [ ]:
df.plot()
pass

In [ ]:
model.evaluate(X_test_s, y_test)

In [ ]:
np.c_[model.predict(X_test_s[:3, :]), y_test[:3]]

In [ ]:
model.save('housing.h5')

In [ ]:
model = keras.models.load_model('housing.h5')

## Tensorflow proability

### Distributions

In [ ]:
[str(x).split('.')[-1][:-2] for x in tfd.distribution.Distribution.__subclasses__()]

In [ ]:
dist = tfd.Normal(loc=100, scale=15)

In [ ]:
x = dist.sample((3,4))
x

In [ ]:
n = 100
xs = dist.sample(n)
plt.hist(xs, density=True)
xp = tf.linspace(50., 150., 100)
plt.plot(xp, dist.prob(xp))
pass

### Broadcasting

In [ ]:
dist = tfd.Normal(loc=[3,4,5,6], scale=0.5)

In [ ]:
dist.sample(5)

In [ ]:
xp = tf.linspace(0., 9., 100)[:, tf.newaxis]
plt.plot(np.tile(xp, dist.batch_shape), dist.prob(xp))
pass

### Mixtures

In [ ]:
tfd.MixtureSameFamily?

In [ ]:
gmm = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
        probs=[0.4, 0.1, 0.2, 0.3]
    ),
    components_distribution=tfd.Normal(
      loc=[3., 4., 5., 6.],      
      scale=[0.1, 0.5, 0.5, .1])
)

In [ ]:
n = 10000
xs = gmm.sample(n)

In [ ]:
sns.distplot(xs)
pass

### Transformations

In [ ]:
[x for x in dir(tfp.bijectors) if x[0].isupper()]

In [ ]:
lognormal = tfp.bijectors.Exp()(tfd.Normal(0, 0.5))

In [ ]:
xs = lognormal.sample(1000)
sns.distplot(xs)
xp = np.linspace(tf.reduce_min(xs), tf.reduce_max(xs), 100)
plt.plot(xp, tfd.LogNormal(loc=0, scale=0.5).prob(xp))
pass

## Regression

In [ ]:
xs = tf.Variable([0., 1., 2., 5., 6., 8.])
ys = tf.sin(xs) + tfd.Normal(loc=0, scale=0.5).sample(xs.shape[0])

In [ ]:
xs.shape, ys.shape

In [ ]:
xs.numpy()

In [ ]:
ys.numpy()

In [ ]:
xp = tf.linspace(-1., 9., 100)[:, None]
plt.scatter(xs.numpy(), ys.numpy())
plt.plot(xp, tf.sin(xp))
pass

In [ ]:
kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(length_scale=1.5)
reg = tfd.GaussianProcessRegressionModel(
    kernel, xp[:, tf.newaxis], xs[:, tf.newaxis], ys
)

In [ ]:
lb.

In [ ]:
ub, lb = reg.mean() + [2*reg.stddev(), -2*reg.stddev()]
plt.fill_between(np.ravel(xp), np.ravel(ub), np.ravel(lb), alpha=0.2)
plt.plot(xp, reg.mean(), c='red', linewidth=2)
plt.scatter(xs[:], ys[:], s=50, c='k')
pass

### Modeling

Sampling from a normal distribuiton using HMC (prior predictive samples)

In [ ]:
[x for x in dir(tfp.mcmc) if x[0].isupper()]

In [ ]:
dir(tfp.vi)

In [ ]:
from tensorflow_probability import edward2 as ed

In [ ]:
# From example in help docs

def unnormalized_log_prob(x):
    return -x - x**2.
 
# Initialize the HMC transition kernel.
num_results = int(1e2)
num_burnin_steps = int(1e2)
adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=unnormalized_log_prob,
        num_leapfrog_steps=3,
        step_size=1.),
    num_adaptation_steps=int(num_burnin_steps * 0.8))
 
# Run the chain (with burn-in).
samples, is_accepted = tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=num_burnin_steps,
    current_state=1.,
    kernel=adaptive_hmc,
    trace_fn=lambda _, pkr: pkr.inner_results.is_accepted)
 
sample_mean = tf.reduce_mean(samples)
sample_stddev = tf.math.reduce_std(samples)

In [ ]:
sample_mean

In [ ]:
sample_stddev

In [ ]:
sns.distplot(samples)
plt.axvline(sample_mean.numpy(), c='red')
plt.plot([sample_mean - 2*sample_stddev, sample_mean + 2*sample_stddev], 
         [0.01, 0.01], c='k', linewidth=3)
pass